# Video Frame Interpolation - Demo Visualizations

This notebook demonstrates the interpolation pipeline and visualizes:
- Input frames
- Interpolated frames
- Simulated adversarial loss curves (synthetic, for demo purposes)


In [ ]:
import sys
from pathlib import Path

sys.path.append(str(Path('..').resolve()))

import numpy as np
import matplotlib.pyplot as plt
import cv2
from src.interpolator import SimpleInterpolator, load_interpolator
from src.simulated_gan_wrapper import SimulatedGANWrapper


## Load Input Frames


In [ ]:
frame1_path = "../data/frame1.jpg"
frame2_path = "../data/frame2.jpg"

frame1 = cv2.imread(frame1_path)
frame2 = cv2.imread(frame2_path)

if frame1 is None or frame2 is None:
    print("Warning: Could not load frames. Creating dummy frames for demo.")
    frame1 = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)
    frame2 = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)

frame1_rgb = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
frame2_rgb = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(frame1_rgb)
axes[0].set_title("Frame 1 (t)")
axes[0].axis('off')
axes[1].imshow(frame2_rgb)
axes[1].set_title("Frame 2 (t+1)")
axes[1].axis('off')
plt.tight_layout()
plt.show()


## Generate Interpolated Frames


In [ ]:
interpolator = load_interpolator(device='cpu')

num_interpolations = 5
interpolated_frames = []

for i in range(1, num_interpolations + 1):
    alpha = i / (num_interpolations + 1)
    print(f"Generating frame with alpha={alpha:.3f}...")
    
    interpolated = interpolator.interpolate(frame1, frame2, alpha)
    interpolated_rgb = cv2.cvtColor(interpolated, cv2.COLOR_BGR2RGB)
    interpolated_frames.append(interpolated_rgb)


## Visualize Interpolated Frames


In [ ]:
all_frames = [frame1_rgb] + interpolated_frames + [frame2_rgb]

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

labels = ["Frame 1 (t)"] + [f"Interpolated {i}" for i in range(1, num_interpolations + 1)] + ["Frame 2 (t+1)"]

for i, (frame, label) in enumerate(zip(all_frames, labels)):
    axes[i].imshow(frame)
    axes[i].set_title(label)
    axes[i].axis('off')

plt.tight_layout()
plt.show()


## Simulated GAN Pipeline (Demo Mode)

**Note**: The following visualizations show **simulated** adversarial losses for demonstration purposes. In a real GAN implementation, these would come from actual discriminator training.


In [ ]:
gan_wrapper = SimulatedGANWrapper(device='cpu')

print("Running simulated GAN pipeline...")
print("NOTE: Discriminator outputs and losses are SIMULATED for demo purposes")

result = gan_wrapper.run_simulation_demo(frame1, frame2, num_interpolations=5)


## Visualize Simulated Adversarial Losses

**Important**: These are synthetic loss curves generated for demonstration. In a real GAN, these would be actual training losses.


In [ ]:
num_epochs = 100
epochs = np.arange(num_epochs)

gen_loss, disc_real, disc_fake = gan_wrapper._generate_synthetic_losses(num_epochs)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(epochs, gen_loss, label='Generator Loss (Simulated)', color='blue', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Simulated Generator Loss (Demo Only)', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].text(0.02, 0.98, 'SIMULATION', transform=axes[0].transAxes,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7),
            fontsize=10, fontweight='bold')

axes[1].plot(epochs, disc_real, label='Discriminator Real Loss (Simulated)', color='green', linewidth=2)
axes[1].plot(epochs, disc_fake, label='Discriminator Fake Loss (Simulated)', color='red', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Simulated Discriminator Losses (Demo Only)', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].text(0.02, 0.98, 'SIMULATION', transform=axes[1].transAxes,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7),
            fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('../outputs/simulated_losses_demo.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nNote: These loss curves are synthetic and generated for demonstration purposes.")
print("In a real GAN implementation, these would be actual training losses from adversarial training.")


In [ ]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

def calculate_metrics(img1, img2):
    if len(img1.shape) == 3:
        img1_gray = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
        img2_gray = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    else:
        img1_gray = img1
        img2_gray = img2
    
    ssim_val = ssim(img1_gray, img2_gray, data_range=255)
    psnr_val = psnr(img1_gray, img2_gray, data_range=255)
    
    return ssim_val, psnr_val

print("Quality Metrics (compared to Frame 1):")
print("-" * 50)
for i, interp_frame in enumerate(interpolated_frames, 1):
    ssim_val, psnr_val = calculate_metrics(frame1_rgb, interp_frame)
    print(f"Interpolated Frame {i}: SSIM={ssim_val:.4f}, PSNR={psnr_val:.2f} dB")
